In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import shutil

import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
# 1 if using cudnn on GPU else using CPU
CUDNN_GPU = 0
model_path = 'model.xlsx'
data_path = '../data/LRCN_dataset.npy'
model_saver = 'model_saver/'
n_classes = 4
n_input = [224,224,3]
n_output = [4]
batch_size = 20

In [3]:
data = np.load(data_path).item()
data['X_train'] = data['X_train']/255
my_model = pd.read_excel(model_path)

In [4]:
# Training Parameters
learning_rate = 0.001
num_steps = 200
#batch_size = 5
display_step = 5

In [5]:
# tf Graph input
X = tf.placeholder(tf.float32, [None, n_input[0], n_input[1], n_input[2]], name='X_train')
Y = tf.placeholder(tf.float32, [None, n_classes], name='y_train')
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [6]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides, padding_):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding=padding_)
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k, strides=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, strides, strides, 1],
                          padding='SAME')


In [7]:
# Create model
def conv_net(x, weights, biases, dropout):

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'],2,'VALID')
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=3)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'],2,'VALID')
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=3)
    
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'],1,'SAME')
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'],1,'SAME')
    conv5 = conv2d(conv4, weights['wc5'], biases['bc5'],1,'SAME')
    conv5 = maxpool2d(conv5, k=3)
    
    
    
    
    

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv5, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Reshape conv2 output to fit fully connected layer input
    fc2 = tf.reshape(fc1, [-1, weights['wd2'].get_shape().as_list()[0]])
    fc2 = tf.add(tf.matmul(fc2, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    
    
    
    # Apply Dropout
    #fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out

In [8]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([7, 7, 3, 96])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 96, 256])),
    
    
    'wc3': tf.Variable(tf.random_normal([3, 3, 256, 384])),
    'wc4': tf.Variable(tf.random_normal([3, 3, 384, 384])),
    'wc5': tf.Variable(tf.random_normal([3, 3, 384, 256])),
    
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*256, 4096])),
    #'wd1': tf.Variable(tf.random_normal([46080, 4096])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd2': tf.Variable(tf.random_normal([4096, 4096])),
    # 1024 inputs, 4 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([4096, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([96])),
    'bc2': tf.Variable(tf.random_normal([256])),
    'bc3': tf.Variable(tf.random_normal([384])),
    'bc4': tf.Variable(tf.random_normal([384])),
    'bc5': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([4096])),
    'bd2': tf.Variable(tf.random_normal([4096])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)


prediction = tf.nn.softmax(logits)
prediction = tf.Print(prediction,[prediction],summarize=20)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
train_len = data['X_train'].shape[0]
test_len = data['X_test'].shape[0]

In [16]:
def get_batch(dict_field_name, batch):
    return data[dict_field_name][batch*batch_size:min((batch+1)*batch_size,train_len)]

In [17]:
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    
    epoch = 0
    
    while True:
        total_batch =  train_len // batch_size
        #for batch in range(total_batch):
        for batch in range(1):
            # lay batch tiep theo
            batch_input = get_batch('X_train', batch) 
            batch_label = get_batch('y_train', batch)
            #print("Y: ", batch_label)
            sess.run(train_op, feed_dict={X: batch_input, Y: batch_label})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_input,
                                                             Y: batch_label})
            print("Epoch " + str(epoch) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))
        epoch += 1
        
            

Epoch 0, Minibatch Loss= 3611527741440.0000, Training Accuracy= 0.450
Epoch 1, Minibatch Loss= 1273572360192.0000, Training Accuracy= 0.450
Epoch 2, Minibatch Loss= 1529691373568.0000, Training Accuracy= 0.200
Epoch 3, Minibatch Loss= 1173914910720.0000, Training Accuracy= 0.800
Epoch 4, Minibatch Loss= 1481788882944.0000, Training Accuracy= 0.750
Epoch 5, Minibatch Loss= 1533512253440.0000, Training Accuracy= 0.800
Epoch 6, Minibatch Loss= 1423031926784.0000, Training Accuracy= 0.800
Epoch 7, Minibatch Loss= 1170163892224.0000, Training Accuracy= 0.850
Epoch 8, Minibatch Loss= 995595845632.0000, Training Accuracy= 0.900
Epoch 9, Minibatch Loss= 848512417792.0000, Training Accuracy= 0.900
Epoch 10, Minibatch Loss= 749369425920.0000, Training Accuracy= 0.950
Epoch 11, Minibatch Loss= 716026413056.0000, Training Accuracy= 0.900
Epoch 12, Minibatch Loss= 709381586944.0000, Training Accuracy= 0.750
Epoch 13, Minibatch Loss= 542539939840.0000, Training Accuracy= 0.900
Epoch 14, Minibatch Lo

KeyboardInterrupt: 

In [ ]:
with tf.Session() as sess:
    pre = sess.run([prediction], feed_dict={X:[data['X_train'][0]]})